In [3]:
import pandas as pd

In [8]:
#This will list all files in the current directory, make sure the COVID files are in the current directory
ls

COVID_Cases_Restricted_Detailed_05172020.csv*
COVID_Cases_Restricted_Detailed_05292020.csv
COVID_Cases_Restricted_Detailed_07312020.csv
COVID_Cases_Restricted_Detailed_08312020.csv
COVID_Cases_Restricted_Detailed_09302020.csv
COVID_Cases_Restricted_Detailed_Internal_06272020.csv
COVID_Data_Explore.ipynb
Explore.ipynb
Untitled.ipynb
Untitled1.ipynb
Untitled2.ipynb
Untitled3.ipynb
combined_covid_data.csv
data_dictionary_covid_cases_restricted_detailed (1).xlsx
summary_data.csv


In [10]:
df_may17 = pd.read_csv('COVID_Cases_Restricted_Detailed_05172020.csv')
df_may29 = pd.read_csv('COVID_Cases_Restricted_Detailed_05292020.csv')
df_june = pd.read_csv('COVID_Cases_Restricted_Detailed_Internal_06272020.csv')
df_july = pd.read_csv('COVID_Cases_Restricted_Detailed_07312020.csv')
df_aug = pd.read_csv('COVID_Cases_Restricted_Detailed_08312020.csv')
df_sep = pd.read_csv('COVID_Cases_Restricted_Detailed_09302020.csv')

In [51]:
#df_june, df_july, df_aug and df_sep have a combined race_ethnicity column, lets break that out

df_june_split = pd.concat([df_june,df_june['race_ethnicity_combined'].str.split(', ', expand = True)], axis = 1)
df_june_split.rename(columns={0: 'race_combined', 1: 'ethnicity'}, inplace=True)

df_july_split = pd.concat([df_july,df_july['race_ethnicity_combined'].str.split(', ', expand = True)], axis = 1)
df_july_split.rename(columns={0: 'race_combined', 1: 'ethnicity'}, inplace=True)

df_aug_split = pd.concat([df_aug,df_aug['race_ethnicity_combined'].str.split(', ', expand = True)], axis = 1)
df_aug_split.rename(columns={0: 'race_combined', 1: 'ethnicity'}, inplace=True)

df_sep_split = pd.concat([df_sep,df_sep['race_ethnicity_combined'].str.split(', ', expand = True)], axis = 1)
df_sep_split.rename(columns={0: 'race_combined', 1: 'ethnicity'}, inplace=True)

#note at the far right new columns for race_combined and ethnicity
df_sep_split.head()

,race_ethnicity_combined,current_status,cdc_report_dt,sex,onset_dt,pos_spec_dt,hosp_yn,icu_yn,death_yn,hc_work_yn,...,headache_yn,abdom_yn,diarrhea_yn,medcond_yn,county_fips_code,res_county,res_state,age_group,race_combined,ethnicity
0,"White, Non-Hispanic",Laboratory-confirmed case,2020-06-15,Female,2020-05-20,2020-06-06,No,Missing,No,No,...,No,No,No,Yes,50001.0,ADDISON,VT,70 - 79 Years,White,Non-Hispanic
1,"White, Non-Hispanic",Laboratory-confirmed case,2020-07-29,Female,2020-07-04,2020-07-12,No,Missing,No,No,...,No,Missing,Yes,Yes,50001.0,ADDISON,VT,70 - 79 Years,White,Non-Hispanic
2,"White, Non-Hispanic",Laboratory-confirmed case,2020-04-20,Female,2020-02-26,2020-03-26,No,Missing,No,No,...,No,No,Yes,Yes,50001.0,ADDISON,VT,70 - 79 Years,White,Non-Hispanic
3,"White, Non-Hispanic",Laboratory-confirmed case,2020-04-21,Female,NaN,NaN,No,Missing,No,No,...,Yes,No,No,Unknown,50001.0,ADDISON,VT,70 - 79 Years,White,Non-Hispanic
4,"White, Non-Hispanic",Laboratory-confirmed case,2020-04-14,Female,2020-04-08,2020-04-10,No,Missing,No,No,...,Yes,No,No,Yes,50001.0,ADDISON,VT,70 - 79 Years,White,Non-Hispanic


In [68]:
#these are all the columns in the june (or later) dataframes that not in the two may dataframes
for x in df_june_split.columns:
    if x not in df_may17.columns or x not in df_may29.columns:
        print(x)

race_ethnicity_combined
county_fips_code
res_county
res_state


In [76]:
#these are all the columns in the May (or later) dataframes that not in the June (or later) dataframes
for x in df_may17.columns:
    if x not in df_june_split.columns:
        print(x)
        
#nothing is printed (all the columns in the df_may17 dataframe are in the df_june_split dataframe)

In [75]:
#just to check all the columns match in the June or later DFs
print(df_june_split.columns == df_july_split.columns)
print(df_july_split.columns == df_aug_split.columns)
print(df_aug_split.columns == df_sep_split.columns)
#it turns out the falses are due to different order not different columns (can use for loop like above to check)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True]
[ True  True  True  True False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True]


In [78]:
#make a list of data frames that get concated
all_dfs = [df_may17,df_may29,df_june_split,df_july_split,df_aug_split,df_sep_split]

working_df = pd.concat(all_dfs)
working_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14077270 entries, 0 to 4481061
Data columns (total 33 columns):
 #   Column                   Dtype 
---  ------                   ----- 
 0   current_status           object
 1   cdc_report_dt            object
 2   sex                      object
 3   age_group                object
 4   race_combined            object
 5   ethnicity                object
 6   onset_dt                 object
 7   pos_spec_dt              object
 8   hosp_yn                  object
 9   icu_yn                   object
 10  death_yn                 object
 11  hc_work_yn               object
 12  pna_yn                   object
 13  abxchest_yn              object
 14  acuterespdistress_yn     object
 15  mechvent_yn              object
 16  fever_yn                 object
 17  sfever_yn                object
 18  chills_yn                object
 19  myalgia_yn               object
 20  runnose_yn               object
 21  sthroat_yn               objec

In [80]:
tr = len(df_may17) + len(df_may29) + len(df_june_split) + len(df_july_split) + len(df_aug_split) + len(df_sep_split)
tr1 = len(df_may17) + len(df_may29) + len(df_june) + len(df_july) + len(df_aug) + len(df_sep)

print(tr)
print(tr1)

14077270
14077270


In [82]:
#taking a look at the data in the new working_df
working_df['race_combined'].value_counts()

Unknown                                   6211151
White                                     2966078
Hispanic/Latino                           2018195
Black                                     1486999
Multiple/Other                             364541
Asian                                      234947
American Indian/Alaska Native               80148
Native Hawaiian/Other Pacific Islander      18664
Name: race_combined, dtype: int64

In [87]:
#Non-hispanic has trailing whitespace maybe?
#this probably won't be useful
working_df['ethnicity'].value_counts()

Non-Hispanic           4171031
Unknown                 721456
Non-Hispanic/Latino     476978
Non-Hispanic            257871
Hispanic/Latino         183843
Not specified            67747
Name: ethnicity, dtype: int64

In [88]:
working_df['sex'].value_counts()

Female     7005397
Male       6561972
Unknown     132642
Missing      90013
Other           79
Name: sex, dtype: int64

In [119]:
working_df['onset_dt'].dtype

dtype('<M8[ns]')

In [112]:
#Change some series data types to datetime
working_df['onset_dt'] = pd.to_datetime(working_df['onset_dt'], yearfirst = True)
working_df['pos_spec_dt'] = pd.to_datetime(working_df['pos_spec_dt'], yearfirst = True)
working_df['cdc_report_dt'] = pd.to_datetime(working_df['cdc_report_dt'], yearfirst = True, utc = True)
working_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14077270 entries, 0 to 4481061
Data columns (total 33 columns):
 #   Column                   Dtype              
---  ------                   -----              
 0   current_status           object             
 1   cdc_report_dt            datetime64[ns, UTC]
 2   sex                      object             
 3   age_group                object             
 4   race_combined            object             
 5   ethnicity                object             
 6   onset_dt                 datetime64[ns]     
 7   pos_spec_dt              datetime64[ns]     
 8   hosp_yn                  object             
 9   icu_yn                   object             
 10  death_yn                 object             
 11  hc_work_yn               object             
 12  pna_yn                   object             
 13  abxchest_yn              object             
 14  acuterespdistress_yn     object             
 15  mechvent_yn              object

In [126]:
#it looks like most of the descriptive data (columns hosp_yn to medcond_yn is missing)
for x in range(8,29):
    print(working_df.iloc[:,x].value_counts())

No         5544364
Unknown    3684594
Missing    3384292
Yes        1464020
Name: hosp_yn, dtype: int64
Missing    7051693
Unknown    5251700
No         1563310
Yes         210567
Name: icu_yn, dtype: int64
No         6096659
Missing    3799822
Unknown    3616829
Yes         563960
Name: death_yn, dtype: int64
Missing    6931999
Unknown    3239944
No         2646194
Yes         455593
Name: hc_work_yn, dtype: int64
Missing    7441398
Unknown    5028844
No         1377519
Yes         229509
Name: pna_yn, dtype: int64
Missing    8273616
Unknown    4952560
No          691811
Yes         152880
Name: abxchest_yn, dtype: int64
Missing    8045028
Unknown    4682189
No         1297198
Yes          52855
Name: acuterespdistress_yn, dtype: int64
Missing    7252618
Unknown    5321277
No         1384983
Yes         118392
Name: mechvent_yn, dtype: int64
Missing    6970185
Unknown    4575672
No         1426495
Yes        1104918
Name: fever_yn, dtype: int64
Missing    7631784
Unknown    4924186
No

In [135]:
#not 100% sure about this
pd_pivot = pd.pivot_table(working_df, columns=['race_combined'], index=['res_state'], aggfunc = 'size')
pd_pivot

race_combined,American Indian/Alaska Native,Asian,Black,Hispanic/Latino,Multiple/Other,Native Hawaiian/Other Pacific Islander,Unknown,White
res_state,,,,,,,,
88,NaN,NaN,NaN,NaN,NaN,5.0,144.0,NaN
99,NaN,NaN,27.0,15.0,NaN,NaN,99.0,99.0
AK,684.0,266.0,157.0,468.0,298.0,5.0,4763.0,2016.0
AL,13.0,328.0,75367.0,18914.0,5507.0,NaN,113901.0,84535.0
AR,NaN,954.0,26416.0,32638.0,890.0,5066.0,19897.0,49604.0
AZ,31906.0,3424.0,14158.0,158564.0,6535.0,543.0,212415.0,106451.0
CA,248.0,29719.0,19754.0,39.0,28043.0,1428.0,827290.0,88647.0
CO,144.0,2352.0,5774.0,56874.0,1641.0,50.0,45272.0,41692.0
CT,NaN,1576.0,16859.0,26784.0,55.0,NaN,55604.0,45454.0


In [136]:
working_df['res_state'].unique()

array([nan, 'LA', 'CA', 'VA', 'ME', 'CO', 'OK', 'TN', 'OH', 'AR', 'NJ',
       'PA', 'GA', 'ID', 'KY', 'MI', 'MA', 'NY', 'FL', 'WI', 'AL', 'SC',
       'IL', 'OR', 'MS', 'WA', 'MN', 'IA', 'KS', 'NC', 'HI', 'MD', 'UT',
       'NV', 'ND', 'AZ', 'NH', 'NE', 'RI', 'MT', 'DE', 'MO', 'TX', 'WY',
       'NYC', 'VT', 'GU', 'NM', 'WV', 'CT', 'AK', 'VI', 'SD', 'IN',
       'OCONUS', 'MP', 'PR', '99', '88', 'USVI', 'DC', 'Unknown'],
      dtype=object)